# 영화 포스터 크롤링

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from urllib import parse

In [2]:
df = pd.read_excel('d:/2nd_project/Data/movie_data/[KOBIS] 박스오피스_줄거리_감정키워드(2003.01~2023.07).xlsx', engine='openpyxl')
df

,영화명,개봉일,누적관객수,등급,장르,대표국적,국적,제작사,배급사,감독,배우,줄거리,키워드
0,#살아있다,2020-06-24,1903992,15세이상관람가,드라마,한국,한국,"영화사 집,(주)퍼스펙티브픽쳐스",롯데컬처웍스(주)롯데엔터테인먼트,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",NaN,NaN
1,#아이엠히어,2021-01-14,16404,12세이상관람가,"멜로/로맨스,코미디",프랑스,프랑스,NaN,(주)넥스트엔터테인먼트월드(NEW),에릭 라티고,"알랭 샤바,배두나,이명자,선종남,김자금,강현중,정윤하,송경의,엄지만,남승화,이지민,...",NaN,NaN
2,#위왓치유,2021-06-03,1908,청소년관람불가,다큐멘터리,기타,기타,NaN,찬란,"바르보라 차르포바,비트 클루삭",NaN,NaN,NaN
3,...ing,2003-11-28,34308,12세관람가,"드라마,멜로/로맨스",한국,한국,드림맥스(주),(주)튜브엔터테인먼트,이언희,"임수정,김래원,이미숙,김지영,김현수,박혜연,장미,이성경,박소연,김순애,최덕문,김인문...",홀엄마가 뽀뽀하자고 달려들면 '욕구 불만이야. 애인을 구해봐.'라고 얘기하는 여고생...,NaN
4,0.0MHz,2019-05-29,137294,15세이상관람가,공포(호러),한국,한국,(주)제이엠컬쳐스,(주)스마일이엔티,유선동,"정은지,이성열,최윤영,신주환,정원창,박명신,남관효,박대희,노혜영,서진석,김대현,신승...","에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,항구의 니쿠코짱!,2023-04-27,10539,전체관람가,애니메이션,일본,일본,NaN,트윈플러스파트너스(주),와타나베 아유무,"오타케 시노부,하나에 나츠키",﻿“보통날이 제일 좋은 거야!” 평범한 인생도 ‘갓생’으로 만드는 무한 긍정 에너지...,NaN
18416,해시태그 시그네,2023-01-11,4411,15세이상관람가,"멜로/로맨스,코미디",노르웨이,노르웨이,NaN,판씨네마(주),NaN,NaN,카페 바리스타로 따분한 인생을 살던 '시그네'에겐 행위 예술가로 매거진 표지를 장식...,NaN
18417,헌티드 맨션,2023-07-26,11075,12세이상관람가,공포(호러),미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,저스틴 시미엔,"티파니 해디쉬,오웬 윌슨,대니 드비토,로자리오 도슨",디즈니 &amp; &lt;캐리비안의 해적&gt; 제작진의 오싹한 초대장! 뉴올리언스...,NaN
18418,호쿠사이,2023-07-16,339,12세이상관람가,드라마,일본,일본,NaN,(주)미디어캐슬,하시모토 하지메,"야기라 유야,타나카 민,타마키 히로시,타키모토 미오리,츠다 칸지,아오키 무네타카",호화찬란한 상인문화로 채색된 에도 거리 한쪽 구석에 싹트지 않는 화가가 한 명이 있...,NaN


In [3]:
df.sort_values(by=['개봉일', '영화명'], ascending=[False, True], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(3)

,영화명,개봉일,누적관객수,등급,장르,대표국적,국적,제작사,배급사,감독,배우,줄거리,키워드
0,레이디,2023-07-26,140,15세이상관람가,"드라마,멜로/로맨스,코미디",한국,한국,(주)크리허브,주식회사 씨엠닉스,오성수,오성수,사연을 품은 채 한적한 마을에서 “파란나라”라는 게스트하우스를 운영하는 주인공 여정...,NaN
1,메타모르포제의 툇마루,2023-07-26,1333,12세이상관람가,기타,일본,일본,NaN,(주)미디어캐슬,카리야마 슌스케,"아시다 마나,미야모토 노부코,타카하시 쿄헤이,후루카와 코토네",인간관계에 서툰 17세 여고생 ‘우라라’.방과 후 서점에서 아르바이트를 하는 그녀의...,NaN
2,밀수,2023-07-26,1978242,15세이상관람가,범죄,한국,한국,(주)외유내강,(주)넥스트엔터테인먼트월드(NEW),류승완,"김혜수,염정아,조인성,박정민,김종수,고민시",열길 물속은 알아도 한길 사람 속은 모른다!평화롭던 바닷가 마을 군천에 화학 공장이...,NaN


In [4]:
new_col = ['검색 영화명' ,'검색 개봉일', '평점', '남자 평점', '여자 평점', '10대 평점', '20대 평점', '30대 평점', '40대 평점', '50대 평점', '리뷰 평점', '리뷰', '포스터']
for col in new_col:
    df[col] = ''
df.head(3)

,영화명,개봉일,누적관객수,등급,장르,대표국적,국적,제작사,배급사,감독,...,남자 평점,여자 평점,10대 평점,20대 평점,30대 평점,40대 평점,50대 평점,리뷰 평점,리뷰,포스터
0,레이디,2023-07-26,140,15세이상관람가,"드라마,멜로/로맨스,코미디",한국,한국,(주)크리허브,주식회사 씨엠닉스,오성수,...,,,,,,,,,,
1,메타모르포제의 툇마루,2023-07-26,1333,12세이상관람가,기타,일본,일본,NaN,(주)미디어캐슬,카리야마 슌스케,...,,,,,,,,,,
2,밀수,2023-07-26,1978242,15세이상관람가,범죄,한국,한국,(주)외유내강,(주)넥스트엔터테인먼트월드(NEW),류승완,...,,,,,,,,,,


In [5]:
df.columns

Index(['영화명', '개봉일', '누적관객수', '등급', '장르', '대표국적', '국적', '제작사', '배급사', '감독',
       '배우', '줄거리', '키워드', '검색 영화명', '검색 개봉일', '평점', '남자 평점', '여자 평점',
       '10대 평점', '20대 평점', '30대 평점', '40대 평점', '50대 평점', '리뷰 평점', '리뷰', '포스터'],
      dtype='object')

In [6]:
df.loc[0]

영화명                                                     레이디
개봉일                                              2023-07-26
누적관객수                                                   140
등급                                                 15세이상관람가
장르                                           드라마,멜로/로맨스,코미디
대표국적                                                     한국
국적                                                       한국
제작사                                                 (주)크리허브
배급사                                               주식회사 씨엠닉스
감독                                                      오성수
배우                                                      오성수
줄거리       사연을 품은 채 한적한 마을에서 “파란나라”라는 게스트하우스를 운영하는 주인공 여정...
키워드                                                     NaN
검색 영화명                                                     
검색 개봉일                                                     
평점                                                         
남자 평점                                   

In [7]:
# num = 0

# for i in df['영화명'][:3]:
#     print(i)
#     df.loc[num, '평점'] = 7
#     # condition = df['영화명'] == i
#     # df.loc[condition, '평점'] = 10
#     num += 1
# df.head(3)

In [8]:
# base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query='
# driver = webdriver.Chrome()
# driver.get(base_url)

# sup_url = parse.quote(f'영화 레이디 평점')
# url = base_url + sup_url
# driver.get(url)
# req = requests.get(url)
# soup = BeautifulSoup(req.text, 'html.parser')

# # 영화명
# mname = movie_list
# print(mname)

# # 기본정보 클릭
# time.sleep(0.3)
# driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[2]/a').click()

# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

In [9]:
# # soup.select('.area_text_title > strong')[0].get_text()
# soup.select('.detail_info > dl')[0].select('div')[0].get_text().strip().split(' ')[1]

In [28]:
base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query='
driver = webdriver.Chrome()
driver.get(base_url)

error_list = []
running_list = []
ec = 0

start_list = list(range(0, 18001, 1000))
end_list = list(range(1000,18421,1000))

for k in range(len(start_list)):
    k = k+1
    df_part = pd.DataFrame(columns = ['영화명', '개봉일', '누적관객수', '등급', '장르', '대표국적', '국적', '제작사', '배급사', '감독', 
    '배우', '줄거리', '키워드', '검색 영화명', '검색 개봉일', '평점', '남자 평점', '여자 평점', 
    '10대 평점', '20대 평점', '30대 평점', '40대 평점', '50대 평점', '리뷰 평점', '리뷰', '포스터'])

    start = start_list[k]
    end = end_list[k]

    count = start

    for movie_list in tqdm(df['영화명'][start:end]):
        error_txt = ''
        try:
            df_part.loc[count, '영화명'] = df.loc[count, '영화명']
            df_part.loc[count, '개봉일'] = df.loc[count,'개봉일']
            df_part.loc[count, '누적관객수'] = df.loc[count,'누적관객수']
            df_part.loc[count, '등급'] = df.loc[count,'등급']
            df_part.loc[count, '장르'] = df.loc[count,'장르']
            df_part.loc[count, '대표국적'] = df.loc[count,'대표국적']
            df_part.loc[count, '국적'] = df.loc[count,'국적']
            df_part.loc[count, '제작사'] = df.loc[count,'제작사']
            df_part.loc[count, '배급사'] = df.loc[count,'배급사']
            df_part.loc[count, '감독'] = df.loc[count,'감독']
            df_part.loc[count, '배우'] = df.loc[count,'배우']
            df_part.loc[count, '줄거리'] = df.loc[count,'줄거리']
            df_part.loc[count, '키워드'] = df.loc[count,'키워드']


            sup_url = parse.quote(f'영화 {movie_list} 정보')
            url = base_url + sup_url
            driver.get(url)
            req = requests.get(url)
            soup = BeautifulSoup(req.text, 'html.parser')
            mname = movie_list

            # 검색 영화명
            try:
                mname = soup.select('.area_text_title > strong')[0].get_text()
                df_part.loc[count, '검색 영화명'] = mname
            except:
                pass

            # 개봉일
            try:
                date = soup.select('.detail_info > dl')[0].select('div')[0].get_text().strip().split(' ')[1]
                df_part.loc[count, '검색 개봉일'] = date
            except:
                df_part.loc[count, '검색 개봉일'] = None
                error_txt = '개봉일 error'

            # 포스터
            try:
                movie_img = soup.select('.detail_info > a')[0].select('img')[0]['src']
                df_part.loc[count, '포스터'] = movie_img
            except:
                df_part.loc[count, '포스터'] = None
                error_txt = '포스터 error'
                    
   
            print(f'{mname} Ok')
        except:
            ec = ec + 1
            print(f'--------{mname} Error------- : {error_txt}')
            error_list.append(mname)
            time.sleep(1)
            continue
        finally:
            count += 1
            if count % 100 == 0:
                driver.quit()
                time.sleep(1)
                driver = webdriver.Chrome()
                driver.get(base_url)
    
    df_part.to_csv(f'd:/2nd_project/Data/poster_test_{k+1}.csv', index=False)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.97 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x01046E73+48323]
	(No symbol) [0x00FD9661]
	(No symbol) [0x00EE5308]
	(No symbol) [0x00F0634C]
	(No symbol) [0x00F02093]
	(No symbol) [0x00F00719]
	(No symbol) [0x00F30DF5]
	(No symbol) [0x00F30ADC]
	(No symbol) [0x00F2A6D6]
	(No symbol) [0x00F0847C]
	(No symbol) [0x00F0957D]
	GetHandleVerifier [0x012AFD5D+2575277]
	GetHandleVerifier [0x012EF86E+2836158]
	GetHandleVerifier [0x012E96DC+2811180]
	GetHandleVerifier [0x010D41B0+626688]
	(No symbol) [0x00FE314C]
	(No symbol) [0x00FDF4B8]
	(No symbol) [0x00FDF59B]
	(No symbol) [0x00FD21B7]
	BaseThreadInitThunk [0x75B900C9+25]
	RtlGetAppContainerNamedObjectPath [0x775F7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x775F7AEE+238]


In [19]:
df_part

,영화명,개봉일,누적관객수,등급,장르,대표국적,국적,제작사,배급사,감독,...,남자 평점,여자 평점,10대 평점,20대 평점,30대 평점,40대 평점,50대 평점,리뷰 평점,리뷰,포스터
0,레이디,2023-07-26,140,15세이상관람가,"드라마,멜로/로맨스,코미디",한국,한국,(주)크리허브,주식회사 씨엠닉스,오성수,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,메타모르포제의 툇마루,2023-07-26,1333,12세이상관람가,기타,일본,일본,NaN,(주)미디어캐슬,카리야마 슌스케,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,밀수,2023-07-26,1978242,15세이상관람가,범죄,한국,한국,(주)외유내강,(주)넥스트엔터테인먼트월드(NEW),류승완,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,붉은 사막,2023-07-26,1293,15세이상관람가,드라마,이탈리아,이탈리아,NaN,일미디어,미켈란젤로 안토니오니,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,비닐하우스,2023-07-26,6597,15세이상관람가,범죄,한국,한국,한국영화아카데미,(주)트리플픽쳐스,이솔희,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,에고이스트,2023-07-26,1217,청소년관람불가,드라마,일본,일본,NaN,"(주)디오시네마,(주)태양미디어그룹",마츠나가 다이시,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,핑크퐁 시네마 콘서트3 : 진저브레드맨을 잡아라,2023-07-26,27919,전체관람가,애니메이션,한국,한국,더핑크퐁컴퍼니(주),더핑크퐁컴퍼니(주),변희선,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,헌티드 맨션,2023-07-26,11075,12세이상관람가,공포(호러),미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,저스틴 시미엔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
8,틴에이지 크라켄 루비,2023-07-25,28242,전체관람가,애니메이션,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),커크 드 미코,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,다크 글래시스,2023-07-21,85,15세이상관람가,"공포(호러),미스터리,스릴러",이탈리아,이탈리아,NaN,주식회사 루믹스미디어,다리오 아르젠토,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
매롱

In [27]:
check = pd.read_csv('d:/2nd_project/Data/poster_test_1.csv')
check[['영화명', '검색 영화명', '검색 개봉일', '포스터']][990:1000]

,영화명,검색 영화명,검색 개봉일,포스터
990,드라이브 마이 카,NaN,NaN,NaN
991,마누라 공유하기 2 - 무삭제판,NaN,NaN,NaN
992,맛있는 치섹,NaN,NaN,NaN
993,오빠 한번 믿어봐 - 무삭제판,NaN,NaN,NaN
994,거유 치나미 사쿠라의 아찔 유혹,NaN,NaN,NaN
995,노트르담,NaN,NaN,NaN
996,매트릭스: 리저렉션,NaN,NaN,NaN
997,새아빠:기막힌 스킬,NaN,NaN,NaN
998,신데렐라 2: 마법에 걸린 왕자,NaN,NaN,NaN
999,아내의 은밀한 과거,NaN,NaN,NaN


In [ ]:
base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query='
driver = webdriver.Chrome()
driver.get(base_url)

error_list = []
ec = 0
count = 0

for movie_list in tqdm(movie_lists):
    try:
        sup_url = parse.quote(f'영화 {movie_list} 평점')
        url = base_url + sup_url
        driver.get(url)
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')

        # 기본정보 클릭
        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[2]/a').click()

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 검색 영화명
        mname = soup.select('.area_text_title > strong')[0].get_text()
        df.loc[count, '검색 영화명'] = mname

        # 개봉일
        date = soup.select('.detail_info > dl')[0].select('div')[0].get_text().strip().split(' ')[1]
        df.loc[count, '검색 개봉일'] = date
        print(date)

        # 포스터
        movie_img = soup.select('.detail_info > a')[0].select('img')[0]['src']
        df.loc[count, '포스터'] = movie_img
        # # 러닝타임
        # runtime = soup.select('.detail_info > dl')[0].select('div')[4].get_text().strip().split(' ')[1]
        
        # # 평점 클릭
        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[4]/a').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 전체 평점
        try:
            tot_score = float(soup.select('.area_star_number')[0].get_text())
            df.loc[count, '평점'] = tot_score
        except:
            df.loc[count, '평점'] = 0

        # 남자 평점
        try:
            man_score = float(soup.select('.area_star_number')[1].get_text())
            df.loc[count, '남자 평점'] = man_score
        except:
            df.loc[count, '남자 평점'] = 0

        # 여자 평점
        try:
            girl_score = float(soup.select('.area_star_number')[2].get_text())
            df.loc[count, '여자 평점'] = girl_score
        except:
            df.loc[count, '여자 평점'] = 0

        # 10대 만족도
        try:
            age10_score = float(soup.select('.area_graph_age > li')[0].select('.this_text_num')[0].get_text())
            df.loc[count, '10대 평점'] = age10_score
        except:
            df.loc[count, '10대 평점'] = 0

        # 20대 만족도
        try:
            age20_score = float(soup.select('.area_graph_age > li')[1].select('.this_text_num')[0].get_text())
            df.loc[count, '20대 평점'] = age20_score
        except:
            df.loc[count, '20대 평점'] = 0

        # 30대 만족도
        try:
            age30_score = float(soup.select('.area_graph_age > li')[2].select('.this_text_num')[0].get_text())
            df.loc[count, '30대 평점'] = age30_score
        except:
            df.loc[count, '30대 평점'] = 0

        # 40대 만족도
        try:
            age40_score = float(soup.select('.area_graph_age > li')[3].select('.this_text_num')[0].get_text())
            df.loc[count, '40대 평점'] = age40_score
        except:
            df.loc[count, '40대 평점'] = 0

        # 50대 만족도
        try:
            age50_score = float(soup.select('.area_graph_age > li')[4].select('.this_text_num')[0].get_text())
            df.loc[count, '50대 평점'] = age50_score
        except:
            df.loc[count, '50대 평점'] = 0


        # 공감순 리뷰 페이지 내리기
        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/div[2]/div/div[1]/ul/li[2]/a').click()

        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/div[4]/div/div/div/div/ul/li[1]/a').click()

        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/div[6]').click()

        webdriver.ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(0.3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(0.3)
        
        # 리뷰 20개 미만 처리
        min = len(soup.select('.area_card_outer._item_wrapper > li'))
        rev_score = []
        rev_row = []

        # 상위 리뷰 3개 추출
        if min >= 3:
            try:
                for cnt in range(0,3):
                    rev = soup.select('.area_card_outer._item_wrapper > li > .area_review_content > div')[cnt].select('.desc')[0].get_text()
                    rev_score = int(soup.select('.area_card_outer._item_wrapper > li .area_text_box')[cnt].get_text().split(')')[1])
                    new_row = pd.DataFrame({'영화명':[mname], '관람객 평점':[tot_score],
                           '남자 평점':[man_score], '여자 평점':[girl_score],
                           '10대 평점':[age10_score], '20대 평점':[age20_score],
                           '30대 평점':[age30_score], '40대 평점':[age40_score],
                           '50대 평점':[age50_score], '리뷰 평점':[rev_score], '리뷰':[rev],
                           '포스터' : [movie_img]})
                    data = pd.concat([data, new_row], axis=0)
                time.sleep(0.3)
            except:
                continue
        else:
            try:
                for cnt in range(0,min):
                    rev = soup.select('.area_card_outer._item_wrapper > li > .area_review_content > div')[cnt].select('.desc')[0].get_text()
                    rev_score = int(soup.select('.area_card_outer._item_wrapper > li .area_text_box')[cnt].get_text().split(')')[1])
                    new_row = pd.DataFrame({'영화명':[mname], '관람객 평점':[tot_score],
                           '남자 평점':[man_score], '여자 평점':[girl_score],
                           '10대 평점':[age10_score], '20대 평점':[age20_score],
                           '30대 평점':[age30_score], '40대 평점':[age40_score],
                           '50대 평점':[age50_score], '리뷰 평점':[rev_score], '리뷰':[rev],
                           '포스터' : [movie_img]})
                    data = pd.concat([data, new_row], axis=0)
                time.sleep(0.3)
            except:
                continue
        time.sleep(1)
    except:
        ec = ec + 1
        print(f'--------{mname} Error-------')
        error_list.append(mname)
        time.sleep(1)
        continue
    finally:
        count += 1
        if count % 100 == 0:
            driver.quit()
            time.sleep(1)
            driver = webdriver.Chrome()
            driver.get(base_url)
        